## VGG16 Pre-trained

In [11]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, Flatten
from sklearn.cross_validation import train_test_split
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras import optimizers
from keras.optimizers import SGD
from keras.models import Model
K.set_floatx('float32')

import pandas as pd
import numpy as np
np.random.seed(17)

train_data = np.load('train.npy')
train_target = np.load('train_target.npy')
train_data = preprocess_input(train_data)
x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.25, random_state=32)

In [16]:
#input = Input(shape=(150, 150, 3))
base_model = VGG16(weights='imagenet', include_top=False)
base_output = base_model.output
x = GlobalAveragePooling2D(name='globalaveragepooling2d')(base_output)
#x = Flatten(name='Flat')(base_output)
x = Dense(4096, activation='relu',name='fc1_Dense')(x)
x = Dropout(0.5, name='dropout_1')(x)
x = Dense(1024, activation='relu',name='fc2_Dense')(x)
x = Dropout(0.5, name='dropout_2')(x)
predictions = Dense(3, activation='softmax',name='output_layer')(x)
model = Model(input=base_model.input, output=predictions)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ou..., inputs=Tensor("in...)`
  # This is added back by InteractiveShellApp.init_path()


In [3]:
# print the model structure
for i, layer in enumerate(model.layers):
    print(i, layer.name)

(0, 'input_1')
(1, 'block1_conv1')
(2, 'block1_conv2')
(3, 'block1_pool')
(4, 'block2_conv1')
(5, 'block2_conv2')
(6, 'block2_pool')
(7, 'block3_conv1')
(8, 'block3_conv2')
(9, 'block3_conv3')
(10, 'block3_pool')
(11, 'block4_conv1')
(12, 'block4_conv2')
(13, 'block4_conv3')
(14, 'block4_pool')
(15, 'block5_conv1')
(16, 'block5_conv2')
(17, 'block5_conv3')
(18, 'block5_pool')
(19, 'globalaveragepooling2d')
(20, 'fc1_Dense')
(21, 'dropout_1')
(22, 'fc2_Dense')
(23, 'dropout_2')
(24, 'output_layer')


In [19]:
# frozen the first 15 layers
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True

# compile the model
sgd = SGD(lr=0.0001, momentum=0.9)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
datagen = ImageDataGenerator(rotation_range=0.9, zoom_range=0.5, shear_range=0.3, \
                             horizontal_flip=False, vertical_flip=True, width_shift_range=0.5)
datagen.fit(train_data)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=15, shuffle=True), steps_per_epoch=len(x_train)/100, \
                    epochs=100, verbose=1, validation_data=(x_val_train, y_val_train))

Epoch 1/100
12/12 [==============================] - 6s - loss: 3.0247 - acc: 0.3000 - val_loss: 1.2202 - val_acc: 0.3820
Epoch 2/100
12/12 [==============================] - 5s - loss: 1.5283 - acc: 0.3611 - val_loss: 1.0637 - val_acc: 0.4331
Epoch 3/100
12/12 [==============================] - 5s - loss: 1.1391 - acc: 0.4778 - val_loss: 1.0082 - val_acc: 0.5280
Epoch 4/100
12/12 [==============================] - 5s - loss: 1.1359 - acc: 0.4500 - val_loss: 1.0112 - val_acc: 0.5280
Epoch 5/100
12/12 [==============================] - 5s - loss: 1.0000 - acc: 0.4833 - val_loss: 1.0133 - val_acc: 0.5158
Epoch 6/100
12/12 [==============================] - 6s - loss: 1.0522 - acc: 0.5111 - val_loss: 1.0096 - val_acc: 0.5231
Epoch 7/100
12/12 [==============================] - 5s - loss: 1.0628 - acc: 0.5194 - val_loss: 1.0100 - val_acc: 0.5280
Epoch 8/100
12/12 [==============================] - 6s - loss: 1.0649 - acc: 0.4889 - val_loss: 1.0095 - val_acc: 0.5280
Epoch 9/100
12/12 [=====

12/12 [==============================] - 6s - loss: 0.9777 - acc: 0.5111 - val_loss: 0.9932 - val_acc: 0.4988
Epoch 68/100
12/12 [==============================] - 6s - loss: 0.9668 - acc: 0.5222 - val_loss: 0.9958 - val_acc: 0.5012
Epoch 69/100
12/12 [==============================] - 6s - loss: 0.9854 - acc: 0.5444 - val_loss: 0.9847 - val_acc: 0.5085
Epoch 70/100
12/12 [==============================] - 5s - loss: 1.0066 - acc: 0.5643 - val_loss: 0.9742 - val_acc: 0.5255
Epoch 71/100
12/12 [==============================] - 6s - loss: 0.9462 - acc: 0.5722 - val_loss: 0.9743 - val_acc: 0.5304
Epoch 72/100
12/12 [==============================] - 6s - loss: 1.0071 - acc: 0.4778 - val_loss: 0.9812 - val_acc: 0.5085
Epoch 73/100
12/12 [==============================] - 6s - loss: 1.0079 - acc: 0.4444 - val_loss: 0.9937 - val_acc: 0.5109
Epoch 74/100
12/12 [==============================] - 6s - loss: 0.9970 - acc: 0.5222 - val_loss: 0.9904 - val_acc: 0.5085
Epoch 75/100
12/12 [=========

In [12]:
test_data = np.load('test.npy')
test_id = np.load('test_id.npy')
test_data = preprocess_input(test_data)
pred = model.predict(test_data)
df = pd.DataFrame(pred, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission.csv', index=False)